In [ ]:
from datasets import load_dataset

cc_news = load_dataset("cc_news")

In [ ]:
from torch.utils.data import DataLoader

loader = DataLoader(cc_news["train"], shuffle=True, batch_size=100)

In [ ]:
samples = next(iter(loader))

len(samples["description"])

In [ ]:
with open("data/cc_news_samples.txt", "w") as f:
    f.write("\n".join([sample for sample in samples["description"] if sample.strip()]))

In [ ]:
daily_dialog = load_dataset("daily_dialog", split="test")

In [ ]:
daily_dialog

In [ ]:
loader = DataLoader(daily_dialog, shuffle=True, batch_size=50)

In [ ]:
daily_dialog[:2]

In [ ]:
dialog_samples = ["".join(dialog) for dialog in daily_dialog[:50]["dialog"]]

In [ ]:
len(dialog_samples)

In [ ]:
with open("data/dialog_samples.txt", "w") as f:
    f.write("\n".join([sample for sample in dialog_samples if sample.strip()]))

In [ ]:
roc_stories = load_dataset("wza/roc_stories")

In [ ]:
roc_stories

In [ ]:
roc_samples = roc_stories["train"][:50]
roc_samples = [" ".join(sample) for sample in zip(roc_samples["sentence1"], roc_samples["sentence2"], roc_samples["sentence3"], roc_samples["sentence4"], roc_samples["sentence5"])]

In [ ]:
roc_samples[:3]

In [ ]:
with open("data/story_samples.txt", "w") as f:
    f.write("\n".join([sample for sample in roc_samples if sample.strip()]))

In [ ]:
import spacy

text = "PersonX falls in love with PersonY"
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

for token in doc:
    print(token.text, token.dep_, [child for child in token.children])

In [ ]:
import string
", short conversation   ".strip(string.punctuation+" ")

In [ ]:
from kogito.core.processors.head import NounPhraseHeadExtractor, VerbPhraseHeadExtractor
import spacy

nlp = spacy.load("en_core_web_sm")

ext = NounPhraseHeadExtractor("ext", nlp)
vext = VerbPhraseHeadExtractor("vext", nlp)
ext.extract("April 24 (Reuters) - Borrowing by U.S. businesses for capital goods rose 2 percent in March from a year earlier, the Equipment Leasing and Finance")

In [ ]:
from kogito.inference import CommonsenseInference

# Initialize inference module with a spacy language pipeline
csi = CommonsenseInference(language="en_core_web_sm")

# Run inference
text = "the researcher"
csi.remove_processor("noun_phrase_extractor")
csi.remove_processor("verb_phrase_extractor")
csi.remove_processor("sentence_extractor")
kgraph = csi.infer(text, extract_heads=True)
print(kgraph)
# Save output knowledge graph to JSON file
# kgraph.to_jsonl("results/test.json")

In [ ]:
from kogito.inference import CommonsenseInference
from kogito.core.knowledge import KnowledgeGraph
from kogito.models.gpt3.zeroshot import GPT3Zeroshot
from kogito.core.relation import KnowledgeRelation, register_relation, EVENT_RELATIONS
import time, os

# csi = CommonsenseInference()
# csi.remove_processor("simple_relation_matcher")

def x_wishes_verbalizer(head, **kwargs):
   # index will be passed from the model
   # so that we can enumerate samples which helps with inference
   index = kwargs.get("index")
   index_txt = f"{index}" if index is not None else ""
   return f"Situation {index_txt}: {head}\nWishes: As a result, PersonX wishes"

X_WISHES = KnowledgeRelation("xWishes",
                             verbalizer=x_wishes_verbalizer,
                             prompt="How does this situation affect each character's wishes?")
register_relation(X_WISHES, kind="social")
print(X_WISHES in EVENT_RELATIONS)

In [ ]:
from kogito.core.knowledge import Knowledge, KnowledgeGraph

kg1 = Knowledge("h1", "xWants", ["hi", "", "  "])
kg2 = Knowledge("h2", "xWants", ["hi", "ho", " hah "])
kg3 = Knowledge("h3", "xWants", ["hi", "", " huha "])
kg4 = Knowledge("h4", "xWants", ["", " ", "  "])

kgraph = KnowledgeGraph([kg1, kg2, kg3, kg4])
print(kgraph)
kgraph.clean()
print(kgraph)

In [ ]:
from kogito.core.utils import SortishSampler

ss = SortishSampler(list(range(100)), 16)

for s in ss:
    print(s)

In [ ]:
from transformers import BertModel

bert = BertModel.from_pretrained("bert-base-uncased")
print(bert)

In [ ]:
from transformers import pipeline

extractor = pipeline("feature-extraction", model="bert-base-uncased")
output = extractor("Hello world", return_tensors="pt")

print(output.shape)

In [ ]:
output

In [ ]:
from transformers import BertLMHeadModel

bert = BertLMHeadModel.from_pretrained("bert-base-uncased")
print(bert)

In [ ]:
from transformers import BartForConditionalGeneration

bart = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
print(bart)

In [8]:
from kogito.core.knowledge import KnowledgeGraph, Knowledge
import pandas as pd

train_graph = KnowledgeGraph.from_csv(
    "data/atomic2020/sample.tsv",
)

In [5]:
import pandas as pd
df = pd.read_csv("data/atomic2020/sample.tsv", header=None)
df.head()

,0,1,2
0,head,relation,tails
1,PersonX abandons ___ altogether,oEffect,none


In [9]:
train_graph[0]

Knowledge(head="PersonX abandons ___ altogether", relation="oEffect", tails=['none'])